In [ ]:
import pandas as pd
from datetime import timedelta, date, datetime
import shutil
import time
from config import crypto
from config import general as config
from finrl.meta.env_custom.random_init import RandomInit
from lib.drl import data_split, train, test, get_model_params
from lib.support import check_run_directory_structure, get_run_timestamp

In [ ]:
def get_dataset_windows(iteration: int = 0):
    start_train = datetime.strptime("11.05.2021", "%d.%m.%Y") 
    start_date = datetime.strptime("25.05.2022", "%d.%m.%Y")
    iterations = []
    if iteration >= crypto.VALIDATION_ITERATIONS:
        raise ValueError(f"max allowed iteration is {crypto.VALIDATION_ITERATIONS - 1}, but {iteration} was provided")
    for i in range(crypto.VALIDATION_ITERATIONS):
        end_date = start_date + timedelta(days=27, hours=23, minutes=59, seconds=59)
        end_train = start_train + timedelta(days=378, hours=23, minutes=59, seconds=59)
        iterations.append({'train_start': start_train, 'train_end': end_train, 'test_start': start_date, 'test_end': end_date})
        # print(f"{i:02} | train: {start_train} - {end_train}  | trade: start: {start_date} end: {end_date}")
        start_date = start_date + timedelta(days=28)
        start_train = start_train + timedelta(days=28)
    iterations = pd.DataFrame(iterations)
    iterations.iloc[6]['test_end'] = '2022-12-01 23:59:59'
    window = {
        "train_start": iterations.iloc[iteration]['train_start'].strftime('%Y-%m-%d'),
        "train_end": iterations.iloc[iteration]['train_end'].strftime('%Y-%m-%d %X'),
        "test_start": iterations.iloc[iteration]['test_start'].strftime('%Y-%m-%d'),
        "test_end": iterations.iloc[iteration]['test_end'].strftime('%Y-%m-%d %X'),
    }
    return window

def get_datasets(iteration):
    dataset_windows = get_dataset_windows(iteration)
    # loading dataset
    df = pd.read_csv(f"{config.DATA_SAVE_DIR}/thesis/crypto_1d_plus.csv", index_col=0)
    train_df = data_split(df, dataset_windows['train_start'], dataset_windows['train_end'])
    test_df = data_split(df, dataset_windows['test_start'], dataset_windows['test_end'])
    print(f"train {train_df.shape} start: {train_df.iloc[0]['date']} end: {train_df.iloc[-1]['date']}")
    print(f"test  {test_df.shape} start: {test_df.iloc[0]['date']} end: {test_df.iloc[-1]['date']}")
    return train_df, test_df

def load_last_state(iteration_results_prefix):
    state_file = pd.read_csv(f"{iteration_results_prefix}_state.csv", index_col=0)
    last_state = state_file.iloc[-1]
    num_shares = last_state[[x+"_amount" for x in TICKER_NAMES]].tolist()
    cash = last_state['cash']
    print(f"cash          : {cash}")
    print(f"NumStockShares: {num_shares}")
    return cash, num_shares

In [ ]:
# global settings
ROOT_DIR = '.'
# check_directory_structure(ROOT_DIR)
STRATEGY_NAME = "cs_eval"
MODEL_NAME = "TD3"
CFG_ID = "V213"
NAME_SUFFIX = "12M"

MODEL_DIR = f"{ROOT_DIR}/{config.TRAINED_MODEL_DIR}/{STRATEGY_NAME}"
TENSORBOARD_DIR = None # No logging for eval runs

INITIAL_MODEL = f"{ROOT_DIR}/trained_models/cs/cs_TD3_V213_12182254_12M"
INITIAL_AMOUNT = 1_000_000

SAVE_MODEL_PATH = f"{ROOT_DIR}/trained_models/{STRATEGY_NAME}"

RETRAIN_TIMESTEPS = 5_000 if MODEL_NAME == "TD3" else 50_000

MODEL_PARAMS = get_model_params(MODEL_NAME, CFG_ID)

RUN_CONFIG = "EVAL"
RUN_NAME_BASE = f"{RUN_CONFIG}_{MODEL_NAME}_{CFG_ID}_{get_run_timestamp()}_{NAME_SUFFIX}"

BASE_RESULTS_DIR = f"{ROOT_DIR}/{config.RESULTS_DIR}/{STRATEGY_NAME}/{MODEL_NAME}/{RUN_NAME_BASE}"

In [ ]:
x, y = get_datasets(0)

In [ ]:
retrain_start = time.time()
for iteration in range(crypto.VALIDATION_ITERATIONS):
    print(f"\nIteration {iteration}")
    print(f"---------------------------------")

    df_train, df_test = get_datasets(iteration)
    TICKER_NAMES = df_test.tic.unique().tolist()

    stock_dimension = len(df_train.tic.unique())
    state_space = 1 + 2 * stock_dimension + len(crypto.INDICATORS_PLUS) * stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

    RUN_NAME = f"{RUN_NAME_BASE}_iter{iteration}"
    SAVE_MODEL_NAME = f"{MODEL_DIR}/{STRATEGY_NAME}_{MODEL_NAME}_{RUN_NAME}"
    RESULTS_FILE_PREFIX = f"{BASE_RESULTS_DIR}/{MODEL_NAME}_{RUN_NAME}"
    PREVIOUS_RESULTS_PREFIX = f"{BASE_RESULTS_DIR}/{MODEL_NAME}_{RUN_NAME_BASE}_iter{iteration-1}"

    if iteration == 0:
        LOAD_MODEL_NAME = f"{INITIAL_MODEL}"
        iteration_start_amount = INITIAL_AMOUNT
        num_stock_shares = [0] * stock_dimension
        do_retrain = False
    else:
        LOAD_MODEL_NAME = f"{MODEL_DIR}/{STRATEGY_NAME}_{MODEL_NAME}_{RUN_NAME_BASE}_iter{iteration-1}"
        iteration_start_amount, num_stock_shares = load_last_state(PREVIOUS_RESULTS_PREFIX)
        do_retrain = True

    ENV_KWARGS = {
        "hmax": 10_000,
        "initial_amount": iteration_start_amount,
        "num_stock_shares": num_stock_shares,
        "buy_cost_pct": [0.001] * stock_dimension,
        "sell_cost_pct": [0.001] * stock_dimension,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": crypto.INDICATORS_PLUS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-6,
        "make_plots": True,
        "mode": "train",
        "strategy_name": STRATEGY_NAME,
        "run_name": RUN_NAME,
        "model_name": MODEL_NAME,
        "random_init": RandomInit(random_init=False, always=False, mod=500, start=100, end=300)
    }

    print(f"Using Model {MODEL_NAME} as {RUN_NAME} with params={MODEL_PARAMS}")
    print(f"Retrain={do_retrain}")
    print(f"Load model from {LOAD_MODEL_NAME}")
    print(f"Save model to   {SAVE_MODEL_NAME}")
    print(f"Save results to {RESULTS_FILE_PREFIX}")

    check_run_directory_structure(ROOT_DIR, config.RESULTS_DIR, STRATEGY_NAME, MODEL_NAME, RUN_NAME_BASE)

    settings = {
        "total_timesteps": RETRAIN_TIMESTEPS,
        "retrain_existing_model": do_retrain,
        "previous_model_name": LOAD_MODEL_NAME,
        "tensorboard_log": TENSORBOARD_DIR,
        "env_kwargs": ENV_KWARGS,
        "model_params": MODEL_PARAMS,
        "save_model": True,
        "target_model_filename": SAVE_MODEL_NAME,
        "file_prefix": RESULTS_FILE_PREFIX
    }
    
    if do_retrain:
        print(f"Retraining on Iteration {iteration}")
        trained = train(df_train, ENV_KWARGS, settings)
    else:
        print(f"Skipping Re-Train for iteration {iteration}")
        shutil.copyfile(LOAD_MODEL_NAME+".zip", SAVE_MODEL_NAME+".zip")
    
    test(df_test, ENV_KWARGS, settings)
    
retrain_end = time.time()
print(f"done all in {retrain_end - retrain_start}")

In [ ]:
# concat separate actions files from each iteration into one file for the whole run
actions = pd.DataFrame()
for it in range(crypto.VALIDATION_ITERATIONS):
    results_base = f"{BASE_RESULTS_DIR}/{MODEL_NAME}_{RUN_NAME_BASE}_iter{it}"
    df_actions_iter = pd.read_csv(f"{results_base}_actions.csv")
    
    # add last day for each iteration 
    last_date = (pd.to_datetime(df_actions_iter.iloc[-1]['date'])+ timedelta(days=1)).strftime('%Y-%m-%d')
    last_row = pd.DataFrame([[last_date] + [0] * stock_dimension + [-1] * stock_dimension])
    last_row.columns = df_actions_iter.columns
    df_actions_iter = pd.concat([df_actions_iter, last_row])
    df_actions_iter['iteration'] = it
    if actions.empty:
        actions = df_actions_iter
    else:
        actions = pd.concat([actions, df_actions_iter])
    # print(results_base)
    
actions = actions.reset_index(drop=True)
actions.to_csv(f"{BASE_RESULTS_DIR}/all_actions.csv", index=False)

In [ ]:
# concat separate state files from each iteration into one file for the whole run
states = pd.DataFrame()
for it in range(crypto.VALIDATION_ITERATIONS):
    results_base = f"{BASE_RESULTS_DIR}/{MODEL_NAME}_{RUN_NAME_BASE}_iter{it}"
    df_state_iter = pd.read_csv(f"{results_base}_state.csv", index_col=0)
    
    df_state_iter['date'] = df_state_iter.index
    df_state_iter = df_state_iter.reset_index(drop=True)
    df_state_iter.loc[0, 'date'] = (pd.to_datetime(df_state_iter.loc[1]['date'])+ timedelta(days=-1)).strftime('%Y-%m-%d')
    df_state_iter = df_state_iter.set_index('date', drop=True)
    df_state_iter['iteration'] = it
    
    if states.empty:
        states = df_state_iter
    else:
        states = pd.concat([states, df_state_iter])
    # print(results_base)
    
# states = states.reset_index(drop=True)
# states.index.rename("date", inplace=True)
states.to_csv(f"{BASE_RESULTS_DIR}/all_state.csv")